In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
import pickle

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [35]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [37]:
import pickle
with open('yelp_dataset.pkl', 'wb') as file:
    pickle.dump(dataset, file)

In [38]:
with open('yelp_dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

In [39]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [40]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data = train_dataset['text']
train_labels = train_dataset['label']
test_data = test_dataset['text']
test_labels = test_dataset['label']

# Convert labels to lists (optional)
# train_labels = train_labels.tolist()
# test_labels = test_labels.tolist()

In [41]:
print(train_data[0])
print(train_labels[0])


dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
4


In [42]:
# print the max sequence length
max_len = max([len(s.split()) for s in train_data])
print(max_len)

1052


In [43]:
max_sequence_length = 1100  # Maximum sequence length
embedding_dim = 128  # Dimension of character embeddings
num_filters = 128  # Number of convolution filters
kernel_size = 5  # Size of convolution kernels
pool_size = 4  # Size of max-pooling window
dropout_rate = 0.5  # Dropout rate
num_classes = len(set(train_labels))  # Number of classes

In [44]:
# train val split
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)

In [45]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(train_data)
sequences_train = tokenizer.texts_to_sequences(train_data)
sequences_val = tokenizer.texts_to_sequences(val_data)
sequences_test = tokenizer.texts_to_sequences(test_data)

In [46]:
X_train = pad_sequences(sequences_train, maxlen=max_sequence_length)
X_val = pad_sequences(sequences_val, maxlen=max_sequence_length)
X_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

In [47]:
y_train = keras.utils.to_categorical(train_labels, num_classes)
y_val = keras.utils.to_categorical(val_labels, num_classes)
y_test = keras.utils.to_categorical(test_labels, num_classes)

In [48]:
# define the model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(input_layer)
conv1d_layer = Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu')(embedding_layer)
maxpool_layer = MaxPooling1D(pool_size=pool_size)(conv1d_layer)
flatten_layer = Flatten()(maxpool_layer)
dropout_layer = tf.keras.layers.Dropout(dropout_rate)(flatten_layer)
output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

In [49]:
model = Model(inputs=input_layer, outputs=output_layer)

In [50]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
num_epochs = 3
batch_size = 128

In [52]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=num_epochs, batch_size=batch_size)

Epoch 1/3
4571/4571 [==============================] - 1124s 246ms/step - loss: 1.1988 - accuracy: 0.4732 - val_loss: 1.0661 - val_accuracy: 0.5358
Epoch 2/3
4571/4571 [==============================] - 1128s 247ms/step - loss: 1.0779 - accuracy: 0.5313 - val_loss: 1.0604 - val_accuracy: 0.5432
Epoch 3/3
 204/4571 [>.............................] - ETA: 17:08 - loss: 1.0415 - accuracy: 0.5522

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test accuracy: ", test_accuracy)
# print classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


In [ ]:
# save the model
model.save('yelp_cnn_model.h5')
torch.save(model.state_dict(), 'yelp_cnn_model.pt')